In [1]:
#Import deps 
import pandas as pd
# ... 

.... Some Introductory Text 

In [3]:
# Import Data 
all_wine_raw_df = pd.read_csv('./Wine_data_both.csv') 
red_wine_df = pd.read_csv('./Wine_data_red.csv')
white_wine_df = pd.read_csv('./Wine_data_white.csv')

In [ ]:
# Preprocess data

In [ ]:
# Build Model 

In [ ]:
# Evaluate and Optimize Model

... Text interpreting and describing results 

In [ ]:
# Export Results